In [1]:
import pandas as pd
import numpy as np

In [2]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/skos/SKOS_NATURE.csv"

skos_natures = pd.read_csv(PATH,header=0)
display(skos_natures)

,skos:Concept,skos:inScheme,skos:broader,skos:closeMatch,skos:prefLabel@fr,skos:altLabel@fr,skos:hiddenLabel,skos:prefLabel@en,skos:altLabel@en,skos:definition
0,Abreuvoir,NatureList,ProprieteNonBatie,NaN,Abreuvoir,NaN,NaN,NaN,NaN,NaN
1,Atelier,NatureList,ProprieteBatie,NaN,Atelier,NaN,NaN,NaN,NaN,NaN
2,Batiment,NatureList,ProprieteBatie,NaN,Bâtiment,NaN,"Bat,Bt",NaN,NaN,NaN
3,Bois,NatureList,ProprieteNonBatie,NaN,Bois,NaN,NaN,NaN,NaN,NaN
4,BoisAgrement,NatureList,"Bois,ObjetDAgrement",Bois,Bois d'agrément,NaN,NaN,NaN,NaN,NaN
5,Canal,NatureList,ProprieteNonBatie,NaN,Canal,NaN,NaN,NaN,NaN,NaN
6,CanalAgrement,NatureList,"Canal,ObjetDAgrement",Canal,Canal d'agrément,NaN,NaN,NaN,NaN,NaN
7,Chemin,NatureList,ProprieteNonBatie,NaN,Chemin,NaN,NaN,NaN,NaN,NaN
8,Cimetiere,NatureList,ProprieteNonBatie,NaN,Cimetiere,NaN,NaN,NaN,NaN,NaN
9,Clos,NatureList,ProprieteNonBatie,NaN,Clos,NaN,NaN,NaN,NaN,NaN


In [3]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastre/Nature/")
g.bind('pnature', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
g.bind('cad', cad)

# Iterate over each row in the DataFrame
for index, row in skos_natures.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.Nature))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:altLabel@fr']):
        if ',' in row["skos:altLabel@fr"]:
            for altLabel in row["skos:altLabel@fr"].split(','):
                g.add((subject_uri, SKOS.altLabel, Literal(altLabel,lang='fr')))
        else:
            g.add((subject_uri, SKOS.altLabel, Literal(altLabel,lang='fr')))
    if row["skos:prefLabel@en"] == row["skos:prefLabel@en"]:
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if row["skos:altLabel@en"] == row["skos:altLabel@en"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if not pd.isnull(row['skos:broader']):
        if ',' in row["skos:broader"]:
            for broader in row["skos:broader"].split(','):
                g.add((subject_uri, SKOS.hiddenLabel, URIRef(baseuri + f"{str(broader)}")))
        else:
            g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if row["skos:closeMatch"] == row["skos:closeMatch"]:
        g.add((subject_uri, SKOS.closeMatch, URIRef(baseuri + f"{str(row['skos:closeMatch'])}")))

    if row["skos:definition"] == row["skos:definition"]:
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition"],lang='fr')))

    if not pd.isnull(row['skos:hiddenLabel']):
        if ',' in row["skos:hiddenLabel"]:
            for hiddenLabel in row["skos:hiddenLabel"].split(','):
                g.add((subject_uri, SKOS.hiddenLabel, Literal(hiddenLabel)))
        else:
            g.add((subject_uri, SKOS.hiddenLabel, Literal(row["skos:hiddenLabel"],lang='fr')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pnature: <http://data.ign.fr/id/codes/cadastre/Nature/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

pnature:Abreuvoir a cad:Nature,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader pnature:ProprieteNonBatie ;
    skos:inScheme cad:NatureList ;
    skos:prefLabel "Abreuvoir"@fr .

pnature:Appenti a cad:Nature,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader pnature:ProprieteBatie ;
    skos:closeMatch pnature:Batiment ;
    skos:inScheme cad:NatureList ;
    skos:prefLabel "Appenti"@fr .

pnature:Atelier a cad:Nature,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader pnature:ProprieteBatie ;
    skos:inScheme cad:NatureList ;
    skos:prefLabel "Atelier"@fr .

pnature:Barraque a cad:Nature,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader pnature:ProprieteBatie ;
    skos:closeMatch pnature